## Phi-4 RAG with Azure AI Search

இந்த நோட்புக் Phi-4-mini மற்றும் Phi-4-multimodal-ஐ Azure AI Search உடன் Retrieval Augmented Generation (RAG) க்கு எவ்வாறு பயன்படுத்துவது என்பதை காட்டுகிறது. இது ஒற்றை-மாதிரி (எழுத்து மட்டும்) மற்றும் பல்மாதிரி (எழுத்து மற்றும் படம்) ஆகிய இரு நிலைகளையும் 포함ிக்கிறது.

**முன் நிபந்தனைகள்:**
*   Azure AI Search vector index (ஒரு எடுக்க உருவாக்க [இந்த அறிவுறுத்தல்களை](https://learn.microsoft.com/azure/search/search-get-started-portal-import-vectors?tabs=sample-data-storage%2Cmodel-aoai%2Cconnect-data-storage) பின்பற்றவும்)
*   Phi-4-mini அல்லது Phi-4-multimodal endpoints Azure AI Foundry இல் அமைக்கப்பட்டிருக்க வேண்டும்


In [ ]:
! pip install azure-ai-inference azure-search-documents

### உரை மட்டமான RAG (Phi-4-mini உடன்)

இந்த பகுதி, உள்ளீடாக வெறும் உரையைப் பயன்படுத்தி, RAGக்காக Phi-4-mini ஐ ஒரு சாட் நிறைவு மாதிரியாய் எப்படி பயன்படுத்துவது என்பதை விளக்குகிறது.  இது Azure AI Foundry Inference மற்றும் Azure AI Search உடன் இணைந்து தொடர்புடைய ஆவணங்களை மீட்டெடுவதையும், மீட்டெடுக்கப்பட்ட சூழ்நிலையைப் பயன்படுத்தி ஒரு பதிலை உருவாக்குவதையும் உள்ளடக்கியது.


In [ ]:
# Step 1: Connect to Azure AI Foundry Inference & Azure AI Search
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import UserMessage
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.models import VectorizableTextQuery

chat_client = ChatCompletionsClient(
    endpoint=os.environ["AZURE_INFERENCE_ENDPOINT"], # Phi-4-mini endpoint
    credential=AzureKeyCredential(os.environ["AZURE_INFERENCE_CREDENTIAL"]),
)

search_client = SearchClient(
    endpoint=os.environ["AZURE_SEARCH_ENDPOINT"],
    index_name=os.environ["AZURE_SEARCH_INDEX"],
    credential=AzureKeyCredential(os.environ["AZURE_SEARCH_KEY"])
)

# Step 2: Retrieve relevant documents from Azure AI Search
def retrieve_documents(query: str, top: int = 10):
    vector_query = VectorizableTextQuery(text=query, k_nearest_neighbors=top, fields="text_vector")
    results = search_client.search(search_text=query,vector_queries=[vector_query],select=["content"],top=top)
    return [doc["content"] for doc in results]

# Step 3: Generate answer using RAG!
def generate_rag_response(query: str):
    docs = retrieve_documents(query)
    context = "\n---\n".join(docs)
    prompt = f"""You are a helpful assistant. Use only the following context to answer the question. If the answer isn't in the context, say 'I don't know'.
    Context: {context} Question: {query} Answer:"""
    response = chat_client.complete(messages=[UserMessage(content=prompt)])
    return response.choices[0].message.content

# Example usage
user_query = "What is the capital of France?"
answer = generate_rag_response(user_query)
print(f"Q: {user_query}\nA: {answer}")


### Phi-4-multimodal உடன் மல்டி-மோடல் RAG

இந்த பகுதி Phi-4-multimodal ஐ RAG க்கான ஒரு chat completion மாதிரியாய் எப்படி பயன்படுத்துவது என்பதை, உரை மற்றும் படம் என்ற இரு வகையான உள்ளீடுகளையும் ஒருங்கிணைப்பதையும் காட்டுகிறது.  இது Azure AI Inference மற்றும் Azure AI Search உடன் இணைதல், தொடர்புடைய ஆவணங்களை மீட்கல், மற்றும் ஒரு மல்டி-மோடல் பதிலை生成 செய்யுதல் ஆகியவற்றைக் கொண்டுள்ளது.

**குறிப்பு:** நீங்கள் Azure AI Search இன்டெக்ஸில் `text_vector` மற்றும் `image_vector` என்ற இரு புலங்களும் இருந்தால், பல-வெக்டர் கேள்வியை மேற்கொள்ளலாம்.


In [ ]:
# Step 1: Connect to Azure AI Inference & Azure AI Search
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import (
    SystemMessage,
    UserMessage,
    TextContentItem,
    ImageContentItem,
    ImageUrl,
    ImageDetailLevel,
)
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.models import VectorizableTextQuery


chat_client = ChatCompletionsClient(
    endpoint=os.environ["AZURE_INFERENCE_ENDPOINT"], #Phi-4-multimodal serverless endpoint
    credential=AzureKeyCredential(os.environ["AZURE_INFERENCE_CREDENTIAL"]),
)

search_client = SearchClient(
    endpoint=os.environ["AZURE_SEARCH_ENDPOINT"],
    index_name=os.environ["AZURE_SEARCH_INDEX"],
    credential=AzureKeyCredential(os.environ["AZURE_SEARCH_KEY"])
)

# Step 2: Retrieve relevant documents from Azure AI Search
def retrieve_documents(query: str, top: int = 3):
    vector_query = VectorizableTextQuery(text=query, k_nearest_neighbors=top, fields="text_vector")
    results = search_client.search(search_text=query, vector_queries=[vector_query], select=["content"], top=top)    
    return [doc["content"] for doc in results]

## Example for Muli-modal search if you have a text_vector AND image_vector field in your vector_index
## NOTE, image vectorization is in preview at the time of writing this code, please use azure-search-documents pypi version >11.6.0b6 
# def retrieve_documents_multimodal(query: str, image_url: str, top: int = 3):
#     text_vector_query = VectorizableTextQuery(
#         text=query,
#         k_nearest_neighbors=top,
#         fields="text_vector",
#         weight=0.5  # Adjust weight as needed
#     )
#     image_vector_query = VectorizableImageUrlQuery(
#         url=image_url,
#         k_nearest_neighbors=top,
#         fields="image_vector",
#         weight=0.5  # Adjust weight as needed
#     )

#     results = search_client.search(
#         search_text=query,  
#         vector_queries=[text_vector_query, image_vector_query],
#         select=["content"],
#         top=top
#     )
#     return [doc["content"] for doc in results]


# Step 3: Generate a multimodal RAG-based answer using retrieved text and an image input
def generate_multimodal_rag_response(query: str, image_url: str):
    # Retrieve text context from search
    docs = retrieve_documents(query)
    context = "\n---\n".join(docs)

    # Build a prompt that combines the retrieved context with the user query
    prompt = f"""You are a helpful assistant. Use only the following context to answer the question. If the answer isn't in the context, say 'I don't know'.
    Context: {context} Question: {query} Answer:"""
    # Create a chat request that includes both text and image input
    response = chat_client.complete(
        messages=[
            SystemMessage(content="You are a helpful assistant that can process both text and images."),
            UserMessage(
                content=[
                    TextContentItem(text=prompt),
                    ImageContentItem(image_url=ImageUrl(url=image_url, detail=ImageDetailLevel.HIGH)),
                ]
            ),
        ]
    )
    return response.choices[0].message.content

# Example usage
user_query = "Can you search for similar items to this shoe?"
sample_image_url = "https://images.unsplash.com/photo-1542291026-7eec264c27ff?q=80&w=1770&auto=format&fit=crop&ixlib=rb-4.0.3"
answer = generate_multimodal_rag_response(user_query, sample_image_url)
print(f"Q: {user_query}\nA: {answer}")

---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
அறிக்குறிப்பு:
இந்த ஆவணம் AI மொழிபெயர்ப்பு சேவையான Co-op Translator (https://github.com/Azure/co-op-translator) மூலம் மொழிபெயர்க்கப்பட்டுள்ளது. நாங்கள் துல்லியத்திற்காக முயற்சி செய்வினும், தானியங்கி மொழிபெயர்ப்புகளில் பிழைகள் அல்லது தவறான தகவல்கள் இருக்கக்கூடும் என்பதை தயவுசெய்து கவனத்தில் கொள்ளவும். அசல் ஆவணம் அதன் தாய்மொழியில் அதிகாரப்பூர்வ ஆதாரமாக கருதப்பட வேண்டும். முக்கியமான தகவல்களுக்கு, தொழில்முறை மனித மொழிபெயர்ப்பாளர் மூலம் மொழிபெயர்ப்பு செய்யப்படும் பரிந்துரைக்கப்படுகிறது. இந்த மொழிபெயர்ப்பினால் ஏற்படும் எந்தவொரு புரிதல் தவறுகளிற்கும் அல்லது தவறான விளக்கங்களுக்காக நாங்கள் பொறுப்பாக இருக்கமாட்டோம்.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
